## Integrantes do grupo:

**(Nome do aluno) - (R.A)**

Luiz Fernando Ferreira Berber - 817123421

Lucas Alves de Sousa - 817116864

Matheus Henrique Silva Paiva - 817123241 

Mateus Sampaio de Souza - 817123420

Pedro Jacó Cunha Mesquita - 817124121

In [1]:
!pip install numpy
!pip install pandas
!pip install regex
!pip install mysql-connector-python

In [2]:
import asyncio
import numpy as np
import pandas as pd
import mysql.connector
import regex

from mysql.connector import Error
from pathlib import Path

In [3]:
def get_files_from_path(filepath):
    path = Path(filepath)
    files = []

    for childs in path.iterdir():
        if childs.is_file(): files.append(childs.parts[len(childs.parts) - 1])

    return files

In [4]:
def create_connection(host_name, user_name, user_password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [5]:
def execute_par_query(connection, query, pars):
    cursor = connection.cursor()
    try:
        cursor.execute(query, pars)
        connection.commit()
        
        if query.lstrip().upper().startswith('INSERT'):
            return cursor.lastrowid
        else:
            return 0
    except Error as e:
        print(f"The error '{e}' occurred")
        print(f'The error query: {query}')

In [6]:
def criar_dataframe_area_atuacao(df_orcamento_despesa):
    return pd.DataFrame({
        'database_id':df_orcamento_despesa['CÓDIGO FUNÇÃO'],
        'cod_funcao':df_orcamento_despesa['CÓDIGO FUNÇÃO'],
        'nome_funcao':df_orcamento_despesa['NOME FUNÇÃO'],
        'cod_sub_funcao':df_orcamento_despesa['CÓDIGO SUBFUNÇÃO'],
        'nome_sub_funcao':df_orcamento_despesa['NOME SUBFUNÇÃO'],
        'valor_orcado':df_orcamento_despesa['ORÇAMENTO INICIAL (R$)']
    })

In [7]:
def criar_dataframe_orgao(df_despesa):
    return pd.DataFrame({
        'database_id':df_despesa['Código Órgão Superior'],
        'cod_orgao_superior':df_despesa['Código Órgão Superior'],
        'nome_orgao_superior':df_despesa['Nome Órgão Superior'],
        'cod_orgao_subordinado':df_despesa['Código Órgão Subordinado'],
        'nome_orgao_subordinado':df_despesa['Nome Órgão Subordinado'],
        'cod_unidade_orcamentaria':df_despesa['Código Unidade Orçamentária'],
        'nome_unidade_orcamentaria':df_despesa['Nome Unidade Orçamentária'],
        'cod_funcao':df_despesa['Código Função'],
        'nome_funcao':df_despesa['Nome Função'],
        'cod_sub_funcao':df_despesa['Código Subfução'],
        'nome_sub_funcao':df_despesa['Nome Subfunção']
    })

In [8]:
def criar_dataframe_programa(df_despesa):
    return pd.DataFrame({
        'database_id':df_despesa['Código Programa Orçamentário'],
        'cod_programa_orcamentario':df_despesa['Código Programa Orçamentário'],
        'nome_programa_orcamentario':df_despesa['Nome Programa Orçamentário'],
        'cod_acao':df_despesa['Código Ação'],
        'nome_acao':df_despesa['Nome Ação'],
        'cod_funcao':df_despesa['Código Função'],
        'nome_funcao':df_despesa['Nome Função'],
        'cod_sub_funcao':df_despesa['Código Subfução'],
        'nome_sub_funcao':df_despesa['Nome Subfunção'],
        'valor_liquidado':df_despesa['Valor Liquidado (R$)']
    })

In [9]:
def criar_dataframe_data(nome_arquivo):
    match = regex.match('(?P<ano>\d{4})(?P<mes>\d{2})', nome_arquivo)
    ano = match.group('ano')
    mes = match.group('mes')
    
    return pd.DataFrame({
        'database_id': [mes],
        'mes':[mes],
        'ano':[ano]
    })

In [10]:
def criar_dataframe_fato():
    return pd.DataFrame({
        'id_dimensao_data': [],
        'id_dimensao_orgao': [],
        'id_dimensao_programa': [],
        'id_dimensao_area_atuacao': [],
        'valor_orcado': [],
        'valor_liquidado': []
    })

In [11]:
def insert_dimensao_area_atuacao(df_dimensao_area_atuacao, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_AREA_ATUACAO(COD_FUNCAO, NOME_FUNCAO, COD_SUB_FUNCAO, NOME_SUB_FUNCAO, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, NOW());"""
    
    for i in range(df_dimensao_area_atuacao.shape[0]):
        if int(df_dimensao_area_atuacao.loc[i]['cod_funcao']) < 0:
            df_dimensao_area_atuacao.at[i, 'cod_funcao'] = 0
        elif int(df_dimensao_area_atuacao.loc[i]['cod_funcao']) == 0:
            df_dimensao_area_atuacao.at[i, 'cod_funcao'] = 1
            
        query_param = (int(df_dimensao_area_atuacao.loc[i]['cod_funcao']), df_dimensao_area_atuacao.loc[i]['nome_funcao'], int(df_dimensao_area_atuacao.loc[i]['cod_sub_funcao']), df_dimensao_area_atuacao.loc[i]['nome_sub_funcao'])
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_area_atuacao.at[i, 'database_id'] = inserted_id

In [12]:
def insert_dimensao_data(df_dimensao_data, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_DATA(MES, ANO, DATA_CRIACAO)
    VALUES(%s, %s, NOW());"""
    
    query_param = (int(df_dimensao_data['mes'][0]), int(df_dimensao_data['ano'][0]))
    inserted_id = execute_par_query(connection, insert_query, query_param)
    
    df_dimensao_data.at[0, 'database_id'] = inserted_id

In [13]:
def insert_dimensao_orgao(df_dimensao_orgao, connection):    
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());"""

    for i in range(df_dimensao_orgao.shape[0]):
        if int(df_dimensao_orgao.loc[i]['cod_orgao_superior']) < 0:
            df_dimensao_orgao.at[i, 'cod_orgao_superior'] = 0
        elif int(df_dimensao_orgao.loc[i]['cod_orgao_superior']) == 0:
            df_dimensao_orgao.at[i, 'cod_orgao_superior'] = 1
            
        query_param = (int(df_dimensao_orgao.loc[i]['cod_orgao_superior']), df_dimensao_orgao.loc[i]['nome_orgao_superior'], int(df_dimensao_orgao.loc[i]['cod_orgao_subordinado']), df_dimensao_orgao.loc[i]['nome_orgao_subordinado'], int(df_dimensao_orgao.loc[i]['cod_unidade_orcamentaria']), df_dimensao_orgao.loc[i]['nome_unidade_orcamentaria'])
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_orgao.at[i, 'database_id'] = inserted_id

In [14]:
def insert_dimensao_programa(df_dimensao_programa, connection):
    insert_query = """
    INSERT INTO ORCAMENTO_PUBLICO.TD_PROGRAMA(Cod_Programa_Orcamentario, Nome_Programa_Orcamentario, Cod_Acao, Nome_Acao, Data_Criacao)
    VALUES(%s, %s, %s, %s, NOW());"""
    
    for i in range(df_dimensao_programa.shape[0]):
        if int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']) < 0:
            df_dimensao_programa.at[i, 'cod_programa_orcamentario'] = 0
        elif int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']) == 0:
            df_dimensao_programa.at[i, 'cod_programa_orcamentario'] = 1
            
        query_param = (int(df_dimensao_programa.loc[i]['cod_programa_orcamentario']), df_dimensao_programa.loc[i]['nome_programa_orcamentario'], df_dimensao_programa.loc[i]['cod_acao'], df_dimensao_programa.loc[i]['nome_acao'])        
        inserted_id = execute_par_query(connection, insert_query, query_param)
        df_dimensao_programa.at[i, 'database_id'] = inserted_id

In [15]:
def unir_dataframes(df_dimensao_data, df_dimensao_orgao, df_dimensao_programa):
    database_id_data = int(df_dimensao_data['database_id'][0])

    df = df_dimensao_orgao.join(df_dimensao_programa, lsuffix='_orgao', rsuffix='_programa')

    df['database_id_data'] = pd.Series(np.random.randn(len(df['cod_funcao_orgao'])), index=df.index)

    for row in range(df.shape[0]):
        df.at[row, 'database_id_data'] = database_id_data

    return df

In [77]:
async def unir_dataframe_fato(df_dimensao_area_atuacao, df_dimensoes):
    size_df = df_dimensao_area_atuacao.shape[0]
    df_fato = pd.DataFrame()
    
    for row in range(size_df):
        print(f'Item {row + 1} de {size_df}')
        
        cod_funcao = int(df_dimensao_area_atuacao.loc[row]['cod_funcao'])
        cod_sub_funcao = int(df_dimensao_area_atuacao.loc[row]['cod_sub_funcao'])
        cod_programa_orcamentario = int(df_dimensoes.loc[row]['cod_programa_orcamentario'])
        
        valor = float(df_dimensao_area_atuacao.loc[0]['valor_orcado'].replace(',', '.'))
        valor_orcado = 0.0 if valor == 0 else valor / 12

        df_dimensoes_por_funcao = df_dimensoes.query(f'cod_funcao_orgao == {cod_funcao} & cod_sub_funcao_orgao == {cod_sub_funcao}')
        
        df_dimensoes_por_funcao['database_id_area_atuacao'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
        df_dimensoes_por_funcao['valor_orcado'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
        
        for row_df in range(df_dimensoes_por_funcao.shape[0]):
            df_dimensoes_por_funcao.iat[row_df, 22] = df_dimensao_area_atuacao.iloc[row][0]
            df_dimensoes_por_funcao.iat[row_df, 23] = valor_orcado
            
        print('Append na fato')
        df_fato = df_fato.append(pd.DataFrame({
            'id_dimensao_data':df_dimensoes_por_funcao['database_id_data'],
            'id_dimensao_orgao':df_dimensoes_por_funcao['database_id_orgao'],
            'id_dimensao_programa':df_dimensoes_por_funcao['database_id_programa'],
            'id_dimensao_area_atuacao':df_dimensoes_por_funcao['database_id_area_atuacao'],
            'valor_orcado':df_dimensoes_por_funcao['valor_orcado'],
            'valor_liquidado':df_dimensoes_por_funcao['valor_liquidado']
        }), ignore_index=True)
        
    return df_fato

In [ ]:
def inserir_fato(df_fato, connection):
    insert_query = """
    INSERT INTO FT_DESPESAS_PUBLICAS(VL_ORCADO, VL_LIQUIDADO, ID_DIMENSAO_PROGRAMA, ID_DIMENSAO_ORGAO, ID_DIMENSAO_AREA_ATUACAO, ID_DIMENSAO_DATA)
    VALUES(%s, %s, %s, %s, %s, %s)"""

    for i in range(df_fato.shape[0]):
        query_param = (df_fato.loc[i]['valor_orcado'], df_fato.loc[i]['valor_liquidado'], df_fato.loc[i]['id_dimensao_programa'], df_fato.loc[i]['id_dimensao_orgao'], df_fato.loc[i]['id_dimensao_area_atuacao'], df_fato.loc[i]['id_dimensao_data'])
        execute_par_query(connection, insert_query, query_param)

In [17]:
mysql_host_name = '127.0.0.1'
mysql_user_name = 'mateus'
mysql_password = 'root123'

In [18]:
path_dados = '../Datas'

df_fato = criar_dataframe_fato()

In [19]:
connection = create_connection(mysql_host_name, mysql_user_name, mysql_password)

Connection to MySQL DB successful


In [20]:
path_orcamento_despesas = f'{path_dados}/OrcamentoDespesas'

arquivos = get_files_from_path(path_orcamento_despesas)

df_orcamento_despesas = pd.read_csv(f'{path_orcamento_despesas}/{arquivos[0]}', sep=';', encoding='mbcs')

In [21]:
df_dimensao_area_atuacao = criar_dataframe_area_atuacao(df_orcamento_despesas)

In [22]:
insert_dimensao_area_atuacao(df_dimensao_area_atuacao, connection)

In [23]:
path_despesa = f'{path_dados}/Despesas'

arquivos = get_files_from_path(path_despesa)

In [24]:
for arquivo in arquivos:    
    print(arquivo)
    
    dados_despesa = pd.read_csv(f'{path_despesa}/{arquivo}', sep=';', encoding='mbcs')
    
    df_dimensao_data = criar_dataframe_data(arquivo)
    df_dimensao_orgao = criar_dataframe_orgao(dados_despesa)
    df_dimensao_programa = criar_dataframe_programa(dados_despesa)
    
    print('Data Frames Criados')
    print('Importando...')
    
    insert_dimensao_data(df_dimensao_data, connection)
    print('Importação dimensão data concluída')
    insert_dimensao_orgao(df_dimensao_orgao, connection)
    print('Importação dimensão orgao concluída')
    insert_dimensao_programa(df_dimensao_programa, connection)
    print('Importação dimensão programa concluída')
    print('Importações Concluídas')
    
    df_dimensoes = unir_dataframes(df_dimensao_data, df_dimensao_orgao, df_dimensao_programa)
    print('Unindo dataframes dimensoes')
    break

201901_Despesas.zip
Data Frames Criados
Importando...
Importação dimensão data concluída
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NO

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

...

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The error '1054 (42S22): Unknown column 'nan' in 'field list'' occurred
The error query: 
    INSERT INTO ORCAMENTO_PUBLICO.TD_ORGAO(COD_ORGAO_SUPERIOR, NOME_ORGAO_SUPERIOR, COD_ORGAO_SUBORDINADO, NOME_ORGAO_SUBORDINADO, COD_UNIDADE_ORCAMENTARIA, NOME_UNIDADE_ORCAMENTARIA, DATA_CRIACAO)
    VALUES(%s, %s, %s, %s, %s, %s, NOW());
The err

In [78]:
df_fato = await unir_dataframe_fato(df_dimensao_area_atuacao, df_dimensoes)

for index in range(df_fato.shape[0]):
    

Item 1 de 26445
Append na fato
Item 2 de 26445
Append na fato
Item 3 de 26445
Append na fato
Item 4 de 26445


<ipython-input-77-15c3d023a187>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dimensoes_por_funcao['database_id_area_atuacao'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)
<ipython-input-77-15c3d023a187>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dimensoes_por_funcao['valor_orcado'] = pd.Series(np.random.randn(len(df_dimensoes_por_funcao['cod_funcao_orgao'])), index=df_dimensoes_por_funcao.index)


Append na fato
Item 5 de 26445
Append na fato
Item 6 de 26445
Append na fato
Item 7 de 26445
Append na fato
Item 8 de 26445
Append na fato
Item 9 de 26445
Append na fato
Item 10 de 26445
Append na fato
Item 11 de 26445
Append na fato
Item 12 de 26445
Append na fato
Item 13 de 26445
Append na fato
Item 14 de 26445
Append na fato
Item 15 de 26445
Append na fato
Item 16 de 26445
Append na fato
Item 17 de 26445
Append na fato
Item 18 de 26445
Append na fato
Item 19 de 26445
Append na fato
Item 20 de 26445
Append na fato
Item 21 de 26445
Append na fato
Item 22 de 26445
Append na fato
Item 23 de 26445
Append na fato
Item 24 de 26445
Append na fato
Item 25 de 26445
Append na fato
Item 26 de 26445
Append na fato
Item 27 de 26445
Append na fato
Item 28 de 26445
Append na fato
Item 29 de 26445
Append na fato
Item 30 de 26445
Append na fato
Item 31 de 26445
Append na fato
Item 32 de 26445
Append na fato
Item 33 de 26445
Append na fato
Item 34 de 26445
Append na fato
Item 35 de 26445
Append na fat

Append na fato
Item 262 de 26445
Append na fato
Item 263 de 26445
Append na fato
Item 264 de 26445
Append na fato
Item 265 de 26445
Append na fato
Item 266 de 26445
Append na fato
Item 267 de 26445
Append na fato
Item 268 de 26445
Append na fato
Item 269 de 26445
Append na fato
Item 270 de 26445
Append na fato
Item 271 de 26445
Append na fato
Item 272 de 26445
Append na fato
Item 273 de 26445
Append na fato
Item 274 de 26445
Append na fato
Item 275 de 26445
Append na fato
Item 276 de 26445
Append na fato
Item 277 de 26445
Append na fato
Item 278 de 26445
Append na fato
Item 279 de 26445
Append na fato
Item 280 de 26445
Append na fato
Item 281 de 26445
Append na fato
Item 282 de 26445
Append na fato
Item 283 de 26445
Append na fato
Item 284 de 26445
Append na fato
Item 285 de 26445
Append na fato
Item 286 de 26445
Append na fato
Item 287 de 26445
Append na fato
Item 288 de 26445
Append na fato
Item 289 de 26445
Append na fato
Item 290 de 26445
Append na fato
Item 291 de 26445
Append na 

Append na fato
Item 511 de 26445
Append na fato
Item 512 de 26445
Append na fato
Item 513 de 26445
Append na fato
Item 514 de 26445
Append na fato
Item 515 de 26445
Append na fato
Item 516 de 26445
Append na fato
Item 517 de 26445
Append na fato
Item 518 de 26445
Append na fato
Item 519 de 26445
Append na fato
Item 520 de 26445
Append na fato
Item 521 de 26445
Append na fato
Item 522 de 26445
Append na fato
Item 523 de 26445
Append na fato
Item 524 de 26445
Append na fato
Item 525 de 26445
Append na fato
Item 526 de 26445
Append na fato
Item 527 de 26445
Append na fato
Item 528 de 26445
Append na fato
Item 529 de 26445
Append na fato
Item 530 de 26445
Append na fato
Item 531 de 26445
Append na fato
Item 532 de 26445
Append na fato
Item 533 de 26445
Append na fato
Item 534 de 26445
Append na fato
Item 535 de 26445
Append na fato
Item 536 de 26445
Append na fato
Item 537 de 26445
Append na fato
Item 538 de 26445
Append na fato
Item 539 de 26445
Append na fato
Item 540 de 26445
Append na 

...

Item 7294 de 26445
Append na fato
Item 7295 de 26445
Append na fato
Item 7296 de 26445
Append na fato
Item 7297 de 26445
Append na fato
Item 7298 de 26445
Append na fato
Item 7299 de 26445
Append na fato
Item 7300 de 26445
Append na fato
Item 7301 de 26445
Append na fato
Item 7302 de 26445
Append na fato
Item 7303 de 26445
Append na fato
Item 7304 de 26445
Append na fato
Item 7305 de 26445
Append na fato
Item 7306 de 26445
Append na fato
Item 7307 de 26445
Append na fato
Item 7308 de 26445
Append na fato
Item 7309 de 26445
Append na fato
Item 7310 de 26445
Append na fato
Item 7311 de 26445
Append na fato
Item 7312 de 26445
Append na fato
Item 7313 de 26445
Append na fato
Item 7314 de 26445
Append na fato
Item 7315 de 26445
Append na fato
Item 7316 de 26445
Append na fato
Item 7317 de 26445
Append na fato
Item 7318 de 26445
Append na fato
Item 7319 de 26445
Append na fato
Item 7320 de 26445
Append na fato
Item 7321 de 26445
Append na fato
Item 7322 de 26445
Append na fato
Item 7323 de 2

Item 7535 de 26445
Append na fato
Item 7536 de 26445
Append na fato
Item 7537 de 26445
Append na fato
Item 7538 de 26445
Append na fato
Item 7539 de 26445
Append na fato
Item 7540 de 26445
Append na fato
Item 7541 de 26445
Append na fato
Item 7542 de 26445
Append na fato
Item 7543 de 26445
Append na fato
Item 7544 de 26445
Append na fato
Item 7545 de 26445
Append na fato
Item 7546 de 26445
Append na fato
Item 7547 de 26445
Append na fato
Item 7548 de 26445
Append na fato
Item 7549 de 26445
Append na fato
Item 7550 de 26445
Append na fato
Item 7551 de 26445
Append na fato
Item 7552 de 26445
Append na fato
Item 7553 de 26445
Append na fato
Item 7554 de 26445
Append na fato
Item 7555 de 26445
Append na fato
Item 7556 de 26445
Append na fato
Item 7557 de 26445
Append na fato
Item 7558 de 26445
Append na fato
Item 7559 de 26445
Append na fato
Item 7560 de 26445
Append na fato
Item 7561 de 26445
Append na fato
Item 7562 de 26445
Append na fato
Item 7563 de 26445
Append na fato
Item 7564 de 2

Item 7776 de 26445
Append na fato
Item 7777 de 26445
Append na fato
Item 7778 de 26445
Append na fato
Item 7779 de 26445
Append na fato
Item 7780 de 26445
Append na fato
Item 7781 de 26445
Append na fato
Item 7782 de 26445
Append na fato
Item 7783 de 26445
Append na fato
Item 7784 de 26445
Append na fato
Item 7785 de 26445
Append na fato
Item 7786 de 26445
Append na fato
Item 7787 de 26445
Append na fato
Item 7788 de 26445
Append na fato
Item 7789 de 26445
Append na fato
Item 7790 de 26445
Append na fato
Item 7791 de 26445
Append na fato
Item 7792 de 26445
Append na fato
Item 7793 de 26445
Append na fato
Item 7794 de 26445
Append na fato
Item 7795 de 26445
Append na fato
Item 7796 de 26445
Append na fato
Item 7797 de 26445
Append na fato
Item 7798 de 26445
Append na fato
Item 7799 de 26445
Append na fato
Item 7800 de 26445
Append na fato
Item 7801 de 26445
Append na fato
Item 7802 de 26445
Append na fato
Item 7803 de 26445
Append na fato
Item 7804 de 26445
Append na fato
Item 7805 de 2

Item 8017 de 26445
Append na fato
Item 8018 de 26445
Append na fato
Item 8019 de 26445
Append na fato
Item 8020 de 26445
Append na fato
Item 8021 de 26445
Append na fato
Item 8022 de 26445
Append na fato
Item 8023 de 26445
Append na fato
Item 8024 de 26445
Append na fato
Item 8025 de 26445
Append na fato
Item 8026 de 26445
Append na fato
Item 8027 de 26445
Append na fato
Item 8028 de 26445
Append na fato
Item 8029 de 26445
Append na fato
Item 8030 de 26445
Append na fato
Item 8031 de 26445
Append na fato
Item 8032 de 26445
Append na fato
Item 8033 de 26445
Append na fato
Item 8034 de 26445
Append na fato
Item 8035 de 26445
Append na fato
Item 8036 de 26445
Append na fato
Item 8037 de 26445
Append na fato
Item 8038 de 26445
Append na fato
Item 8039 de 26445
Append na fato
Item 8040 de 26445
Append na fato
Item 8041 de 26445
Append na fato
Item 8042 de 26445
Append na fato
Item 8043 de 26445
Append na fato
Item 8044 de 26445
Append na fato
Item 8045 de 26445
Append na fato
Item 8046 de 2

MemoryError: Unable to allocate 225. MiB for an array with shape (4, 7383564) and data type float64

In [80]:
test.head()

,id_dimensao_data,id_dimensao_orgao,id_dimensao_programa,id_dimensao_area_atuacao,valor_orcado,valor_liquidado
0,32.0,967269.0,956688,221401.0,0.0,"0,0000"
1,32.0,969866.0,959310,221401.0,0.0,"0,0000"
2,32.0,975915.0,965409,221401.0,0.0,"0,0000"


In [ ]:
inserir_fato(test, connection)